# Investor - Flow of Funds - US

### Introduction:

Special thanks to: https://github.com/rgrp for sharing the dataset.

### Step 1. Import the necessary libraries

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f37c6d5a25a5dde94f02a29cec326788e333afe3fc30c2087171d582326e4d56
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import expr, col, mean, when, sum, count, desc, min, max

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv).

In [4]:
!wget https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv

--2024-04-15 21:17:48--  https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2315 (2.3K) [text/plain]
Saving to: ‘weekly.csv’

weekly.csv          100%[===================>]   2.26K  --.-KB/s    in 0s      

2024-04-15 21:17:48 (37.3 MB/s) - ‘weekly.csv’ saved [2315/2315]



### Step 3. Assign it to a variable called

In [5]:
df = spark.read.csv('weekly.csv', inferSchema=True, header=True, sep=',')

In [6]:
df.show(5)

+----------+------------+---------------+------------+------+----------+------------+--------------+-----+
|      Date|Total Equity|Domestic Equity|World Equity|Hybrid|Total Bond|Taxable Bond|Municipal Bond|Total|
+----------+------------+---------------+------------+------+----------+------------+--------------+-----+
|2012-12-05|       -7426|          -6060|       -1367|   -74|      5317|        4210|          1107|-2183|
|2012-12-12|       -8783|          -7520|       -1263|   123|      1818|        1598|           219|-6842|
|2012-12-19|       -5496|          -5470|         -26|   -73|       103|        3472|         -3369|-5466|
|2012-12-26|       -4451|          -4076|        -375|   550|      2610|        3333|          -722|-1291|
|2013-01-02|      -11156|          -9622|       -1533|  -158|      2383|        2103|           280|-8931|
+----------+------------+---------------+------------+------+----------+------------+--------------+-----+
only showing top 5 rows



### Step 4.  What is the frequency of the dataset?

### Step 5. Set the column Date as the index.

### Step 6. What is the type of the index?

### Step 7. Set the index to a DatetimeIndex type

### Step 8.  Change the frequency to monthly, sum the values and assign it to monthly.

In [13]:
monthly = df.groupBy(F.year('Date').alias("year"), F.month('Date').alias("month")).sum().orderBy('year', 'month')

In [14]:
monthly.show()

+----+-----+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|year|month|sum(Total Equity)|sum(Domestic Equity)|sum(World Equity)|sum(Hybrid)|sum(Total Bond)|sum(Taxable Bond)|sum(Municipal Bond)|sum(Total)|
+----+-----+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|2012|   12|           -26156|              -23126|            -3031|        526|           9848|            12613|              -2765|    -15782|
|2013|    1|             3661|               -1627|             5288|       2730|          12149|             9414|               2735|     18540|
|2014|    4|            10842|                1048|             9794|       4931|           8493|             7193|               1300|     24267|
|2014|    5|            -2203|               -8720|             6518|       3172|          13767|            10192|   

### Step 9. You will notice that it filled the dataFrame with months that don't have any data with NaN. Let's drop these rows.

In [17]:
# Not in PySpark

### Step 10. Good, now we have the monthly data. Now change the frequency to year.

In [15]:
yearly = monthly.groupBy("year").sum().orderBy('year')

In [16]:
yearly.show()

+----+---------+----------+----------------------+-------------------------+----------------------+----------------+--------------------+----------------------+------------------------+---------------+
|year|sum(year)|sum(month)|sum(sum(Total Equity))|sum(sum(Domestic Equity))|sum(sum(World Equity))|sum(sum(Hybrid))|sum(sum(Total Bond))|sum(sum(Taxable Bond))|sum(sum(Municipal Bond))|sum(sum(Total))|
+----+---------+----------+----------------------+-------------------------+----------------------+----------------+--------------------+----------------------+------------------------+---------------+
|2012|     2012|        12|                -26156|                   -23126|                 -3031|             526|                9848|                 12613|                   -2765|         -15782|
|2013|     2013|         1|                  3661|                    -1627|                  5288|            2730|               12149|                  9414|                    2735|       

### BONUS: Create your own question and answer it.